In [ ]:
import requests
import json
import string
base_url = 'http://localhost:3000/'
#base_url = 'https://seekbeta.lisym.org/'

API_TOKEN = open("./files/token").readline().strip() #"user:password" encoded in base64
headers = {"Authorization": "Basic %s" %API_TOKEN,
           "Content-type": "application/json",
           "Accept": "application/json"}
file_upload_headers = headers.copy()
file_upload_headers.pop('Content-type');

In [ ]:
def print_response(response_text):
    if (r.ok):
        obj = json.loads(response_text)
        print("Returned successfully posted object:\n",obj)
        return obj
    else:
        print("Error: ",r.text)
    return None

In [ ]:
def init_json_data(file, title=None):
    json_format = json.load(open(file))
    if (title is not None):
        json_format['data']['attributes']['title'] = title
    json_data = json.dumps(json_format)
    return json_data

In [ ]:
#delete previous projects
#for i in range (2,7):
#    url = base_url + "projects/%i.json" %i
#    r = requests.delete(url, headers=headers)
#delete data files
#for i in range(7,13):
#    url = base_url + "data_files/%i.json" %i
#    r = requests.delete(url, headers=headers)

### Post a project

In [ ]:
proj_file = "./json/post_max_project.json"
json_data = init_json_data(proj_file, "Project LEDA")
url = base_url + "projects"
print(json_data)
r = requests.post(url, headers=headers, data=json_data)
proj = print_response(r.text)

### Make current user a member of the created project - currently needs to be done manually?! 

### Post an Investigation based on the posted project

In [ ]:
inv_file = "./json/post_max_investigation.json"
url = base_url + "investigations"
json_data = init_json_data(inv_file, "API test Investigation")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
inv = print_response(r.text)

### Post a Study based on the Investigation 

In [ ]:
study_file = "./json/post_max_study.json"
url = base_url + "studies"
json_data = init_json_data(study_file, "API test Study" )
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "INV_ID", inv['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
study = print_response(r.text)

### Post an Experimental Assay based on the Study

In [ ]:
assay_file = "./json/post_max_assay.json"
url = base_url + "assays"
json_data = init_json_data(assay_file, "API Corrected Experimental assay")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "STUDY_ID", study['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
assay = print_response(r.text)

### Post a Data File linked to the posted Assay

In [ ]:
#First POST the meta-data
df_file = "./json/post_max_data_file.json"
url = base_url + "data_files"
json_data = init_json_data(df_file, "API Test Data File")
json_data = str.replace(json_data, "PROJECT_ID",proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
print(json_data)
print(headers)
r = requests.post(url, headers=headers, data=json_data)

df = print_response(r.text) #only metadata object
actual_file = "./files/meltdown.pdf"
# #Second, add the file (PUT)
url = df['data']['attributes']['content_blobs'][0]['link']  # content_blobs array size = 1 
r = requests.put(url, headers=file_upload_headers, data=open(actual_file, 'rb'))
print("Response Status:",r)

### Post an SOP linked to an Assay

In [ ]:
#First POST the meta-data
sop_file = "./json/post_max_sop.json"
url = base_url + "sops"
json_data = init_json_data(sop_file, "API Test SOP")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
print(json_data)
r = requests.post(url, headers=headers, data=json_data)
sop = print_response(r.text)
##Second, add the file (PUT)
url = sop['data']['attributes']['content_blobs'][0]['link']  #only one file in an Sop 
r = requests.put(url, headers=file_upload_headers, data=open('./files/meltdown.pdf', 'rb'))
print("Response Status: ",r)

### Post a Document linked to an Assay

In [ ]:
#First POST the meta-data
doc_file = "./json/post_max_document.json"
url = base_url + "documents"
json_data = init_json_data(doc_file, "API Test Document")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
doc = print_response(r.text)
#Second, add the file (PUT)
url = doc['data']['attributes']['content_blobs'][0]['link']  #only one file in an Sop 
r = requests.put(url, headers=file_upload_headers, data=open('./files/meltdown.pdf', 'rb'))
print("Response status: ", r)

### Post a Model linked to an Assay

In [ ]:
#First POST the meta-data
model_file = "./json/post_max_model.json"
url = base_url + "models"
json_data = init_json_data(model_file, "API Test Model")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
mod = print_response(r.text)

#Second, add the file (PUT)
blob_url = lambda bi: mod['data']['attributes']['content_blobs'][bi]['link']
blob_type = lambda bi: str.replace(mod['data']['attributes']['content_blobs'][bi]['content_type'], "application/", "")
filename = lambda bi: "./files/model_"+str(bi+1)+"."+blob_type(bi)

for i in range(len(mod['data']['attributes']['content_blobs'])):
    print("uploading", filename(i), "to", blob_url(i))
    r = requests.put(blob_url(i), headers=file_upload_headers, data=open(filename(i), 'rb'))
    print("Response Status:", r)

### Post a Presentation (linked to a Project)

In [ ]:
#First POST the meta-data
pr_file = "./json/post_max_presentation.json"
url = base_url + "presentations"
json_data = init_json_data(pr_file, "API Test Presentation")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
pr = print_response(r.text)
#Second, add the file (PUT)
url = pr['data']['attributes']['content_blobs'][0]['link']  #only one file in a presentation
r = requests.put(url, headers=file_upload_headers,  data=open('./files/presentation.pdf', 'rb'))
print("Response Status: ", r)

### Post a Modeling Assay with linked Assets 

In [ ]:
assay_file2 = "./json/post_max_assay2.json"
url = base_url + "assays"
json_data = init_json_data(assay_file2, "API Test Modeling Assay with linked assets")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "STUDY_ID", study['data']['id'])
json_data = str.replace(json_data, "SOP_ID", sop['data']['id'])
json_data = str.replace(json_data, "MODEL_ID", mod['data']['id'])
json_data = str.replace(json_data, "DOCUMENT_ID", doc['data']['id'])
json_data = str.replace(json_data, "DATAFILE_ID", df['data']['id'])

r = requests.post(url, headers=headers, data=json_data)
assay2 = print_response(r.text)

### Post a DataFile with remote content (URL) --> similar for other assets

In [ ]:
df_file = "./json/post_remote_data_file.json"
url = base_url + "data_files"
json_data = init_json_data(df_file, "API remote Data File")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)

remote_df = print_response(r.text) 

### Post a Model with remote content

In [ ]:
model_file = "./json/post_remote_model.json"
url = base_url + "models"
json_data = init_json_data(model_file, "API remote model example")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)

remote_model = print_response(r.text) 